In [2]:
import seaborn as sns
sns.set_theme()

# Load the iris dataset from seaborn
iris = sns.load_dataset("iris")

# Filter the dataset to only include 'versicolor' and 'setosa' species
filtered_iris = iris[iris['species'].isin(['versicolor', 'setosa'])]

# Select only 'sepal_length' and 'sepal_width' variables
filtered_iris = filtered_iris[['sepal_length', 'sepal_width', 'species']]

# Label 'setosa' as 1 and 'versicolor' as -1
filtered_iris['species'] = filtered_iris['species'].map({'setosa': 1, 'versicolor': -1})

sepal_size_inputs = filtered_iris[['sepal_length', 'sepal_width']].to_numpy()
target_species_output = filtered_iris['species'].to_numpy()

In [ ]:
import numpy as np
import warnings
#234567891123456789212345678931234567894123456789512345678961234567897123456789
#                                python docstring limit: 72 characters |      
#                                            python code limit: 79 characters |
# test change
class SingleNeuron(object):
    """
    A class used to represent a single neuron (in the machine learning 
    sense).

    Design notes: the model type and the activation function are 
    intended to be immutable, though of course python still lets you mutate them.
    """
    type_perceptron = "perceptron"
    type_linear_regression_1D = "linear regression 1D"

    def sign(input_value):
        """ 

        """
        if input_value >= 0:
            return 1
        else: 
            return 0
        
    def linear_1D(input_value):
        """ 

        """
        return input_value
    
    # default_activation_function = sign

    # def test_for_matching_ndarrays(var_1, var_2):
    #     return type(var_1) != np.ndarray or type(var_2) != np.ndarray \
    #            or var_1.shape != var_1.shape

    def perceptron_loss_function(predicted_outputs, 
                                 target_outputs):
        """ 
        
        """
        # if SingleNeuron.test_for_matching_ndarrays(predicted_values, true_values) == False:
        #     raise TypeError
        return (1/4) * np.sum((predicted_outputs - target_outputs)**2)
    
    def perceptron_stochastic_gradient(predicted_output, 
                                       target_output):
        """ 
        
        """
        return (1/2) * (predicted_output - target_output)

    def linear_regression_loss_function(predicted_outputs, 
                                        target_outputs):
        """ 
        
        """
        # if SingleNeuron.test_for_matching_ndarrays(predicted_values, true_values) == False:
        #     raise TypeError
        return (1/(2*target_outputs.size)) * np.sum((predicted_outputs - target_outputs)**2)
        
    def linear_regression_1D_stochastic_gradient(predicted_output, 
                                                 target_output, 
                                                 training_data_length):
        """ 
        
        """
        return (1/training_data_length) * (predicted_output - target_output)

    def preactivation(input, weights, bias):
        """ 
        
        """
        assert np.shape(input) is np.shape(weights), \
                "Input vector must have the same shape as weights vector." \
                + f"{np.shape(input) = },  {np.shape(weights) = }"
        return np.dot(input, weights) + bias
    
    def __init__(self, 
                 data_dimension, 
                 model_type, 
                 weights=None, 
                 bias=None, 
                 activation_function=None):
        """
        
        """
        self.data_dimension = data_dimension 
        # A single number for each feature vector has dimension 1, 
        # a 2D vector has dimension 2, etc.

        self.model_type = model_type

        if activation_function is None:
            if self.model_type == SingleNeuron.type_perceptron:
                self.activation_function = SingleNeuron.sign
            elif self.model_type == SingleNeuron.type_linear_regression_1D:
                self.activation_function = SingleNeuron.linear_1D
        else:
            self.activation_function = activation_function
        
        if weights is None:
            self.weights = np.random.randn(data_dimension)
        else:
            self.weights = weights

        if bias is None:
            self.bias = np.random.randn()
        else:
            self.bias = bias

    def predict_outputs(self, 
                        inputs, 
                        weights=None, 
                        bias=None, 
                        use_current_weights_and_bias=True):
        """ 
        
        """
        if use_current_weights_and_bias:
            weights = self.weights
            bias = self.bias

        assert (np.isscalar(inputs) and self.data_dimension == 1) \
                or ((not np.isscalar(inputs)) and inputs.shape[-1] == self.data_dimension), \
                    "Prediction: mismatch between expected feature vector " \
                        + f"\ndimension ({self.data_dimension = }) and input " \
                        + f"shape ({np.shape(inputs) = })."
        
        if ((not np.isscalar(inputs)) and len(inputs.shape) == 1):
            return self.activation_function(SingleNeuron.preactivation(inputs, weights, bias))
        else:
            return [self.activation_function(SingleNeuron.preactivation(input, weights, bias)) for input in inputs]
    
    def current_weights(self):
        """ 
        
        """
        return self.weights.copy()
    
    def current_bias(self):
        """ 
        
        """
        return self.bias

    def current_weights_and_bias(self):
        """ 
        
        """
        return (self.weights.copy(), self.bias)
    
    def perceptron_stochastic_gradient_update(self, 
                                              input, 
                                              target_output, 
                                              learning_rate=None):
        """ 
        learning_rate does nothing and is just for making it more uniform to pass around update functions
        """
        gradient = SingleNeuron.perceptron_stochastic_gradient(self.predict_outputs(input), target_output)
        self.weights -= gradient * input
        self.bias -= gradient
        return gradient

    def linear_regression_1D_stochastic_gradient_update(self, 
                                                        input, 
                                                        target_output, 
                                                        learning_rate):
        gradient = SingleNeuron.linear_regression_1D_stochastic_gradient(self.predict_outputs(input), target_output)
        self.weights -= learning_rate * gradient * input
        self.bias -= learning_rate * gradient
        return gradient
        
    def train(self, 
              inputs, 
              target_outputs, 
              learning_rate=0.5, 
              num_epochs=50):
        """ 
        
        """
        # if self.model_type != SingleNeuron.type_perceptron and (learning_rate == None or epochs == None):
        #     raise ValueError("learning_rate and epochs must be specified for non-perceptron models")
        weight_bias_update = None
        loss_function = None
        if self.model_type is SingleNeuron.type_perceptron:
            weight_bias_update = self.perceptron_stochastic_gradient_update
            loss_function = SingleNeuron.perceptron_loss_function
        elif self.model_type is SingleNeuron.type_linear_regression_1D:
            weight_bias_update = self.linear_regression_1D_stochastic_gradient_update
            loss_function = SingleNeuron.linear_regression_loss_function

        loss_at_epoch = [-1] * (num_epochs + 1)
        loss_at_epoch[0] = loss_function(self.predict_outputs(inputs), target_outputs)

        for epoch_index in range(num_epochs):
            for input, target_output in zip(inputs, target_outputs):
                weight_bias_update(input, target_output, learning_rate)
            loss_at_epoch[epoch_index+1] = loss_function(self.predict_outputs(inputs), target_outputs)

# todo
    def __repr__(self):
        """ 
        
        """
        return self
    
# might not be necessary
    # def __call__(self):
    #     return self

In [57]:
neuron_test = SingleNeuron(sepal_size_inputs.shape[1], "perceptron")

In [44]:
print(f"{len(np.shape(np.transpose(sepal_size_inputs[1]))) = }")
print(f"{np.shape(np.transpose(sepal_size_inputs)) = }")
print(f"{len(sepal_size_inputs) = }")
print(f"{type(sepal_size_inputs) = },  {type(5) = },  {type(5.1) = }")

len(np.shape(np.transpose(sepal_size_inputs[1]))) = 1
np.shape(np.transpose(sepal_size_inputs)) = (2, 100)
len(sepal_size_inputs) = 100
type(sepal_size_inputs) = <class 'numpy.ndarray'>,  type(5) = <class 'int'>,  type(5.1) = <class 'float'>


In [58]:
print(f"{neuron_test.current_weights_and_bias() = }")
[current_weights, current_bias] = neuron_test.current_weights_and_bias()
print(f"{sepal_size_inputs[1] = }")
print(f"{SingleNeuron.preactivation(sepal_size_inputs[1], current_weights, current_bias) = }")
preactivation_value = SingleNeuron.preactivation(sepal_size_inputs[1], current_weights, current_bias)
print(f"{neuron_test.activation_function(preactivation_value) = }")
print(f"{neuron_test.predict_outputs(sepal_size_inputs[1]) = }")
inputs = sepal_size_inputs
activation_values = [neuron_test.activation_function(SingleNeuron.preactivation(input, current_weights, current_bias)) for input in inputs]
print(f"{activation_values = }")
print(f"{neuron_test.predict_outputs(inputs) = }")

neuron_test.current_weights_and_bias() = (array([-1.04808694,  2.89509232]), -2.0549427790680297)
sepal_size_inputs[1] = array([4.9, 3. ])
SingleNeuron.preactivation(sepal_size_inputs[1], current_weights, current_bias) = np.float64(1.494708141533946)
neuron_test.activation_function(preactivation_value) = 1
neuron_test.predict_outputs(sepal_size_inputs[1]) = 1
activation_values = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1]
neuron_test.predict_outputs(inputs) = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,

In [ ]:
[(SingleNeuron.sign(SingleNeuron.preactivation(input, neuron_test.current_weights(), neuron_test.current_bias()))) for input in sepal_size_inputs[1]]

In [ ]:
SingleNeuron.sign(np.float64(3.3034947687319463))

In [ ]:
print(f"{SingleNeuron.sign(0) = }")
print(f"{SingleNeuron.linear_1D(8.3) = }")

In [ ]:
print(f"{sepal_size_inputs.shape = }")
print(f"{sepal_size_inputs.shape[1] = }")
print(f"{sepal_size_inputs[1] = }")

In [ ]:
x = np.random.randn(3)
type(x) == np.ndarray

In [ ]:
y = np.random.randn(4)

In [ ]:
np.dot(x,y)

In [ ]:
np.dot(2,4)

In [ ]:
for xval in x:
    print(xval)